# <div style="text-align: center; color: cyan">Train</div>

## <div style="text-align: center; color: lime">Imports</div>

In [1]:
import torch
from torch import nn
from torch.optim import Adam
from torch.utils.data import Dataset, DataLoader, random_split

from sklearn.datasets import load_iris

## <div style="text-align: center; color: lime">AutoGrad</div>

In [2]:
a = torch.tensor(3.0, requires_grad=True)
b = torch.tensor(2.0, requires_grad=True)

y = a ** 2 + b

In [3]:
y.backward()

print("dy/da: ", a.grad.item())  # d(a**2 + b)/da = 2*a ----a=3----> 6
print("dy/db: ", b.grad.item())  # d(a**2+b)/db = 1

dy/da:  6.0
dy/db:  1.0


In [4]:
w = torch.tensor(5.0, requires_grad=True)  # weight
b = torch.tensor(2.0, requires_grad=True)  # bias

x = 2  # input
y_true = 7  # true output

y_hat = w * x + b  # prediction

loss = (y_hat - y_true) ** 2  # calculate loss
loss.backward()  # calculate gradients

print(f"d(loss)/dw: {w.grad.item()}")
print(f"d(loss)/db: {b.grad.item()}")



d(loss)/dw: 20.0
d(loss)/db: 10.0


## <div style="text-align: center; color: lime">Loss Function</div>

In [5]:
y_true = torch.tensor([0, 1])
y = torch.tensor([
    [2.0, 8.0],
    [5.0, 5.0],
])

loss_fn = nn.CrossEntropyLoss()
loss = loss_fn(y, y_true)

print(loss.item())

3.347811460494995


In [6]:
y_true = torch.tensor([0, 1])
y = torch.tensor([
    [100.0, 0.0],
    [0.0, 100.0]
])

loss_fn = nn.CrossEntropyLoss()
loss = loss_fn(y, y_true)

print(loss.item())


0.0


## <div style="text-align: center; color: lime">Optimizer</div>

In [7]:
model = nn.Linear(4, 2)

optimizer = Adam(model.parameters())

In [8]:
x = torch.tensor([
    [1.0, 2.0, 3.0, 4.0],
    [-1.0, -2.0, -3.0, -4.0],
])  # simple data
y_true = torch.tensor([0, 1])  # simple targe

for step in range(10):
    optimizer.zero_grad()  # clear the gradients

    logits = model(x)  # make a prediction

    loss = loss_fn(logits, y_true)  # calculate the loss
    print(f"step {step}, loss: {loss.item()}")

    loss.backward()  # calculate the gradients with respect to loss

    optimizer.step()  # optimize the weights

step 0, loss: 0.8765422701835632
step 1, loss: 0.8651807308197021
step 2, loss: 0.8539098501205444
step 3, loss: 0.842731773853302
step 4, loss: 0.8316479921340942
step 5, loss: 0.8206604719161987
step 6, loss: 0.8097708225250244
step 7, loss: 0.7989808320999146
step 8, loss: 0.7882919311523438
step 9, loss: 0.7777056097984314


## <div style="text-align: center; color: lime">Load the data and make the model </div>

In [9]:
iris = load_iris()

data = torch.tensor(iris.data).to(torch.float)
target = torch.tensor(iris.target)

In [10]:
class IRISDataset(Dataset):
    def __init__(self, data, target):
        super().__init__()
        self.data = data
        self.target = target

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.target[idx]


iris_dataset = IRISDataset(data, target)

In [11]:
g1 = torch.Generator().manual_seed(20)
train_data, val_data, test_data = random_split(iris_dataset, [0.7, 0.2, 0.1], g1)

train_loader = DataLoader(train_data, batch_size=10, shuffle=True)
val_loader = DataLoader(val_data, batch_size=10, shuffle=False)
test_loader = DataLoader(test_data, batch_size=10, shuffle=False)

In [12]:
class IRISClassifier(nn.Module):
    def __init__(self):
        super().__init__()

        self.layers = nn.Sequential(
            nn.Linear(4, 16),
            nn.Linear(16, 8),
            nn.Linear(8, 3),
        )

    def forward(self, x):
        return self.layers(x)


iris_classifier = IRISClassifier()

## <div style="text-align: center; color: lime">Train the model</div>

In [13]:
loss_fn = nn.CrossEntropyLoss()
optimizer = Adam(iris_classifier.parameters())

In [14]:
for batch_of_data, batch_of_target in train_loader:
    optimizer.zero_grad()

    logits = iris_classifier(batch_of_data)

    loss = loss_fn(logits, batch_of_target)
    print(f"loss: {loss.item()}")

    loss.backward()

    optimizer.step()


loss: 1.181538462638855
loss: 1.1570122241973877
loss: 1.1441924571990967
loss: 1.1753343343734741
loss: 1.1002519130706787
loss: 1.1666862964630127
loss: 1.0838695764541626
loss: 1.1226308345794678
loss: 1.1205450296401978
loss: 1.1404510736465454
loss: 1.094001054763794


<div style="text-align: center">

<div>
    @LiterallyTheOne — PhD Candidate in Artificial Intelligence
</div>

<a style="margin: 1em" href="https://literallytheone.github.io">
https://literallytheone.github.io
</a>

</div>
